# Object detection using Transfer Learning of CNN architectures



1.   Load in a pre-trained CNN model trained on a large dataset
2.   Freeze parameters(weights) in the model’s lower convolutional layers
3.   Add a custom classifier with several layers of trainable parameters to model
4.   Train classifier layers on training data available for the task
5.   Fine-tune hyperparameters and unfreeze more layers as needed




In [ ]:
from keras.applications import ResNet50
from keras.layers import Flatten, Dense, Dropout
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_path = '/content/drive/My Drive/DL_Hussain_Khambati_33537/rural_and_urban_photos/train'
valid_path = '/content/drive/My Drive/DL_Hussain_Khambati_33537/rural_and_urban_photos/val'

In [ ]:
# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Freeze early layers
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Add custom classifier
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)

In [ ]:
# Create full model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
# Create generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(224, 224),
        batch_size=2,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        valid_path,
        target_size=(224, 224),
        batch_size=2,
        class_mode='categorical')

In [ ]:
# Train classifier layers
model.fit(
        train_generator,
        steps_per_epoch=5,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=50)

In [ ]:
# Unfreeze deeper layers
for layer in base_model.layers[-6:]:
    layer.trainable = True

# Recompile and fine-tune model
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(
        train_generator,
        steps_per_epoch=5,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=50)